# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
from keras import backend as K
from sklearn.utils import class_weight as cw

class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename, encoding= 'unicode_escape')
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['text'], float):
                texts.append(str(row['text']))
            else:
                texts.append(row['text'])

            labels.append(row['sentiment'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = tf.keras.preprocessing.sequence.pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_metric(self, y_true, y_pred):
        accuracyScore = accuracy_score(y_true, y_pred)

        # binary: Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.
        precisionScoreBinary = precision_score(y_true, y_pred, average='binary')
        recallScoreBinary = recall_score(y_true, y_pred, average='binary')
        f1ScoreBinary = f1_score(y_true, y_pred, average='binary')

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def print_metric(self, accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary):
        print("Accuracy: " + str(accuracyScore))
        print("Precision: " + str(precisionScoreBinary))
        print("Recall: " + str(recallScoreBinary))
        print("F1-Score: " + str(f1ScoreBinary))
        print(str(accuracyScore) + "," + str(precisionScoreBinary) + "," + str(recallScoreBinary) + "," + str(
            f1ScoreBinary))

    def get_testing_metric(self, y_test, y_pred):
        # metric for Testing Data
        # print("Testing Data")
        accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = self.get_metric(y_test, y_pred)
        # print()

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_val_data(self, textsTraining, labelsTraining, train_index, val_index, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining)[train_index], np.asarray(textsTraining)[val_index]
        y_train, y_val = np.asarray(labelsTraining)[train_index], np.asarray(labelsTraining)[val_index]

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)

    def recall_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self, y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    def get_weight(self, y):
        class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
        return class_weight_current

Using TensorFlow backend.


# Finite State Machine

In [2]:
import random

def FSM():
    fsm = {}
    fsm[0] = {'src': 0, 'dst': 1, 'layer': 'embedding_layer', 'next_path': [1]}
    fsm[1] = {'src': 1, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[2] = {'src': 2, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[3] = {'src': 2, 'dst': 3, 'layer': 'maxpooling_layer', 'next_path': [4]}
    fsm[4] = {'src': 3, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[5] = {'src': 2, 'dst': 4, 'layer': 'global_maxpooling_layer', 'next_path': [6, 7]}
    fsm[6] = {'src': 4, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[7] = {'src': 4, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[8] = {'src': 5, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[9] = {'src': 5, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[10] = {'src': 6, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[11] = {'src': 5, 'dst': 7, 'layer': 'output_layer', 'next_path': []}
    fsm[12] = {'src': 6, 'dst': 7, 'layer': 'output_layer', 'next_path': []}

    return fsm

def mutateFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'after': ['convolutional_layer'], 'change': 'maxpooling_layer'}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'after': ['convolutional_layer'], 'change': 'convolutional_layer'}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'after': ['dense_layer'], 'change': 'dropout_layer'}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'after': ['dense_layer'], 'change': 'dense_layer'}

    return fsm

def addFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'add': ['convolutional_layer', 'maxpooling_layer']}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'add': ['convolutional_layer']}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'add': ['dense_layer', 'dropout_layer']}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'add': ['dense_layer']}

    return fsm

def addConvLayer(idx, hyperparameters):
    hyperparameters['num_filters' + str(idx)] = random.randint(32, 512)
    hyperparameters['kernel_size' + str(idx)] = random.randint(1, 5)
    hyperparameters['conv_activation_func' + str(idx)] = random.choice(['relu', 'softmax', 'elu', 'selu',
                                                'softplus', 'softsign', 'tanh',
                                                'sigmoid', 'hard_sigmoid', 'linear'])
    hyperparameters['conv_init_mode' + str(idx)] = random.choice([
                                          'zeros',
                                          'ones',
                                          'uniform',
                                          'normal',
                                          'glorot_normal',
                                          'glorot_uniform',
                                          'he_normal',
                                          'he_uniform',
                                          'lecun_normal',
                                          'lecun_uniform'])
    hyperparameters['conv_weight_constraint' + str(idx)] = random.randint(1, 5)


def addDenseLayer(idx, hyperparameters):
    hyperparameters['neurons' + str(idx)] = random.randint(1, 30)
    hyperparameters['dense_activation_func' + str(idx)] = random.choice(['relu', 'softmax', 'elu', 'selu',
                                                'softplus', 'softsign', 'tanh',
                                                'sigmoid', 'hard_sigmoid', 'linear'])
    hyperparameters['dense_init_mode' + str(idx)] = random.choice([
                                          'zeros',
                                          'ones',
                                          'uniform',
                                          'normal',
                                          'glorot_normal',
                                          'glorot_uniform',
                                          'he_normal',
                                          'he_uniform',
                                          'lecun_normal',
                                          'lecun_uniform'])
    hyperparameters['dense_weight_constraint' + str(idx)] = random.randint(1, 5)


def addMaxPoolingLayer(idx, hyperparameters):
    hyperparameters['pool_size' + str(idx)] = random.randint(2, 6)


def addDropoutLayer(idx, hyperparameters):
    hyperparameters['dropout_rate' + str(idx)] = random.uniform(0, 1)

def getLayerSize(layer, conv_idx, dense_idx, dropout_idx, maxpooling_idx):
    if layer == 'convolutional_layer':
        conv_idx += 1
    elif layer == 'dense_layer':
        dense_idx += 1
    elif layer == 'dropout_layer':
        dropout_idx += 1
    elif layer == 'maxpooling_layer':
        maxpooling_idx += 1
    return conv_idx, dense_idx, dropout_idx, maxpooling_idx


def addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, hyperparameters):
    idx = 0
    while idx < max_conv_idx:
        idx += 1
        addConvLayer(idx, hyperparameters)

    idx = 0
    while idx < max_dense_idx:
        idx += 1
        addDenseLayer(idx, hyperparameters)

    idx = 0
    while idx < max_maxpooling_idx:
        idx += 1
        addMaxPoolingLayer(idx, hyperparameters)

    idx = 0
    while idx < max_dropout_idx:
        idx += 1
        addDropoutLayer(idx, hyperparameters)


def generateFSM(hyperparameters):
    fsm = FSM()

    idx = conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
    path = [fsm[idx]['layer']]
    while len(fsm[idx]['next_path']) != 0:
        idx = random.choice(fsm[idx]['next_path'])
        layer = fsm[idx]['layer']
        path.append(layer)
        conv_idx, dense_idx, dropout_idx, maxpooling_idx = getLayerSize(layer, conv_idx, dense_idx, dropout_idx,
                                                                        maxpooling_idx)

    addLayerToolboxes(conv_idx, dense_idx, dropout_idx, maxpooling_idx, hyperparameters)
    hyperparameters['layers'] = path

# CNN

In [3]:
import tensorflow as tf

class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

        return model

# Path

In [4]:
training_path = 'train.csv'
val_path = 'validation.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = '{}/Dataset/Twitter US Airline Sentiment/'.format(root_path)
resultsPath = '{}/Server1/Twitter US Airline Sentiment/Paper DGGA-CNN/Results/'.format(root_path)
archPath = '{}/Server1/Twitter US Airline Sentiment/Paper DGGA-CNN/Architecture/'.format(root_path)
testing_name = 'RandomSearch'
glovePath = ['{}/Glove/glove.6B.50d.txt'.format(root_path),
             '{}/Glove/glove.6B.100d.txt'.format(root_path),
             '{}/Glove/glove.6B.200d.txt'.format(root_path),
             '{}/Glove/glove.6B.300d.txt'.format(root_path)]

# Run

In [5]:
import time
import os
import csv
import tensorflow as tf
import random
from time import sleep

# object class
util = utility()
cnn = CNN()

# Read data
dfTraining = util.read_CSV(datasetPath + training_path)
dfTesting = util.read_CSV(datasetPath + val_path)

# get texts and labels
textsTraining, y_train = util.get_text_label(dfTraining)
textsTesting, y_test = util.get_text_label(dfTesting)

# Tokenize words
tokenizer = util.tokenize_texts(textsTraining)
X_train = tokenizer.texts_to_sequences(textsTraining)
X_test = tokenizer.texts_to_sequences(textsTesting)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# get maxlen
maxlen = util.get_max_length_of_sentences(textsTraining)

# Pad sequences with zeros
X_train = util.padding_texts(X_train, maxlen)
X_test = util.padding_texts(X_test, maxlen)

tempAcc= 0.8433632095603927
for i in range(2877, 3000):
    text = ''
    hyperparameters = {}
    
    # Optimized hyperparameters
    hyperparameters['epochs'] = random.randint(1, 100)
    hyperparameters['batch_size'] = random.randint(32, 256)
    hyperparameters['optimizer'] = random.choice(['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam'])
    hyperparameters['learning_rate'] = random.uniform(1e-4, 1e-2)
    hyperparameters['momentum'] = random.uniform(0, 1)
    hyperparameters['output_init_mode'] = random.choice(['zeros',
                              'ones',
                              'uniform',
                              'normal',
                              'glorot_normal',
                              'glorot_uniform',
                              'he_normal',
                              'he_uniform',
                              'lecun_normal',
                              'lecun_uniform'])
    hyperparameters['output_dim'] = random.choice([50, 100, 200, 300])
    
    generateFSM(hyperparameters)

    # print(hyperparameters)

    if int(hyperparameters['output_dim']) == 50:
        glove = glovePath[0]
    elif int(hyperparameters['output_dim']) == 100:
        glove = glovePath[1]
    elif int(hyperparameters['output_dim']) == 200:
        glove = glovePath[2]
    elif int(hyperparameters['output_dim']) == 300:
        glove = glovePath[3]

    embedding_matrix = util.create_embedding_matrix(glove, tokenizer.word_index, int(hyperparameters['output_dim']))

    then = time.time()
    # grid search
    model = cnn.cnn_model(vocab_size, maxlen, embedding_matrix, hyperparameters, hyperparameters['layers'])

    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', verbose=False, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(archPath + testing_name + str(i + 1) + ".h5", monitor='val_acc', mode='max', verbose=False, save_best_only=True)]
    
    # model.summary()
    y_train = np.uint8(y_train)
    y_test = np.uint8(y_test)
    model.fit(X_train, y_train, epochs=hyperparameters['epochs'], verbose=False, validation_data=(X_test, y_test),
              batch_size=hyperparameters['batch_size'], callbacks=callbacks, class_weight=util.get_weight(y_train))

    # load the saved model
    for x in range(0, 4):  # try 4 times
        try:
            # msg.send()
            saved_model = tf.keras.models.load_model(archPath + testing_name + str(i + 1) + ".h5")   
            str_error = None
        except Exception as e:
            print('An error occurs when loading saved model.')
            str_error = e
            pass

        if str_error:
            sleep(2)  # wait for 2 seconds before trying to fetch the data again
        else:
            break
    
    y_pred = saved_model.predict_classes(X_test)
    os.remove(archPath + testing_name + str(i + 1) + ".h5")   

    # CNN metrics
    accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = util.get_testing_metric(y_test, y_pred)

    now = time.time()
    diff = now - then
    # print(diff)
    # print(accuracyScore)

    # save testing data
    f = open(resultsPath + testing_name + ".csv", 'a')
    text ='i,accuracy,precision,recall,f1Score,time,'
    for param in hyperparameters:
        text += "{0},".format(param)
    
    text += "\n{0},{1},{2},{3},{4},{5}".format(str(i + 1),accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary, diff)
    for value in hyperparameters.values():
        text += ",{0}".format(value)
    text += '\n'    
    f.write(text)    
    f.close()

    # save testing data
    f = open(resultsPath + testing_name + 'summary' + ".csv", 'a')
    text = "{0},{1},{2},{3},{4},{5}\n".format(str(i + 1),accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary, diff)
    if accuracyScore > tempAcc:
        tempAcc = accuracyScore
        print(tempAcc)
    f.write(text)    
    f.close() 

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [ ]:
%%javascript
Jupyter.notebook.session.delete(); 

<IPython.core.display.Javascript object>